In [31]:
import transformers
import torch
import pandas as pd
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, Trainer, TrainingArguments, BartForConditionalGeneration
from transformers.modeling_outputs import BaseModelOutput
from torch.optim import Adam
from accelerate import Accelerator
import wandb
from tqdm import tqdm
from torch.optim import AdamW

In [32]:
from evaluate import load
from rouge_score import rouge_scorer
from bert_score import score as bert_score

# meteor = load("meteor")
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [33]:
accelerator = Accelerator()

In [34]:
!wandb login --relogin --verify ''

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/dhruv/.netrc
wandb: Currently logged in as: noel22338 (nlp_project_team) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [35]:
training_data = pd.read_csv('../Dataset/train.csv')
testing_data = pd.read_csv('../Dataset/test.csv')
validation_data = pd.read_csv('../Dataset/validation.csv')

In [36]:
training_data.head()

,hatespeech,csType,counterspeech,Suggest,Relevance,Aggressive,Complexity,Comments,source,claim,...,hatespeechTarget,powerDynamics,prompt_offensiveness,prompt_target_group,prompt_speaker_intent,prompt_power_dynamics,prompt_implication,prompt_emotional_reaction,prompt_cognitive_reaction,prompt_cs_generation
0,Maybe the UN could talk to those asian and afr...,Informative,The us is the second most polluting country in...,3,4.0,2.0,3.0,NaN,Human,The UN should focus on Asian and African natio...,...,asian_people,racial,Analyze the offensiveness of the statement: Ma...,Identify the group of people that the speaker ...,Analyze the speaker's intention behind writing...,Explain the underlying power dynamics between ...,Explain the implied meaning underlying the off...,Describe how the target group might feel emoti...,Describe how the target group might react cogn...,Analyze the different aspects such as offensiv...
1,Maybe the UN could talk to those asian and afr...,Questioning,Doesn't everyone on the planet have a responsi...,,3.0,2.0,2.0,NaN,Human,The UN should focus on Asian and African natio...,...,asian_people,racial,Analyze the offensiveness of the statement: Ma...,Identify the group of people that the speaker ...,Analyze the speaker's intention behind writing...,Explain the underlying power dynamics between ...,Explain the implied meaning underlying the off...,Describe how the target group might feel emoti...,Describe how the target group might react cogn...,Analyze the different aspects such as offensiv...
2,Maybe the UN could talk to those asian and afr...,Denouncing,The world would be a better place if people we...,1,1.0,2.0,2.0,NaN,Human,The UN should focus on Asian and African natio...,...,asian_people,racial,Analyze the offensiveness of the statement: Ma...,Identify the group of people that the speaker ...,Analyze the speaker's intention behind writing...,Explain the underlying power dynamics between ...,Explain the implied meaning underlying the off...,Describe how the target group might feel emoti...,Describe how the target group might react cogn...,Analyze the different aspects such as offensiv...
3,Maybe the UN could talk to those asian and afr...,Positive,"You're right, ocean pollution is one of the ma...",3,4.0,2.0,2.0,NaN,Human,The UN should focus on Asian and African natio...,...,asian_people,racial,Analyze the offensiveness of the statement: Ma...,Identify the group of people that the speaker ...,Analyze the speaker's intention behind writing...,Explain the underlying power dynamics between ...,Explain the implied meaning underlying the off...,Describe how the target group might feel emoti...,Describe how the target group might react cogn...,Analyze the different aspects such as offensiv...
4,Juice Jews are worse that nukes. Caption to a ...,Informative,Anti-semitism is a serious problem that we nee...,2,3.0,2.0,2.0,NaN,Human,Jews are worse than nukes.,...,jews,religion,Analyze the offensiveness of the statement: Ju...,Identify the group of people that the speaker ...,Analyze the speaker's intention behind writing...,Explain the underlying power dynamics between ...,Explain the implied meaning underlying the off...,Describe how the target group might feel emoti...,Describe how the target group might react cogn...,Analyze the different aspects such as offensiv...


In [37]:
columns = training_data.columns
print(columns)

Index(['hatespeech', 'csType', 'counterspeech', 'Suggest', 'Relevance',
       'Aggressive', 'Complexity', 'Comments', 'source', 'claim',
       'centralTopic', 'speakerIntent', 'targetGroup', 'relevantPowerDynamics',
       'hatespeechImplication', 'targetGroupEmotionalReaction',
       'targetGroupCognitiveReaction', 'hatespeechOffensiveness', 'id',
       'is_high_quality', 'hs_id', 'hatespeechTarget', 'powerDynamics',
       'prompt_offensiveness', 'prompt_target_group', 'prompt_speaker_intent',
       'prompt_power_dynamics', 'prompt_implication',
       'prompt_emotional_reaction', 'prompt_cognitive_reaction',
       'prompt_cs_generation'],
      dtype='object')


In [38]:
class DialoGPTDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.tokenizer = AutoTokenizer.from_pretrained("GroNLP/hateBERT")
        self.bart_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        # Tokenize hate speech
        hate_inputs = self.tokenizer(
            row["hatespeech"],
            return_tensors='pt',
            max_length=128,
            truncation=True,
            padding="max_length"
        )
        # Tokenize counterspeech
        counter_inputs = self.bart_tokenizer(
            row["counterspeech"],
            return_tensors='pt',
            max_length=128,
            truncation=True,
            padding="max_length"
        )

        # Define intent categories
        categories = {
            'informative': 0,
            'questioning': 1,
            'denouncing': 2,
            'positive': 3,
            'humor': 4
        }

        return {
            'input_ids': hate_inputs['input_ids'].squeeze(0),
            'attention_mask': hate_inputs['attention_mask'].squeeze(0),
            'counter_speech': counter_inputs['input_ids'].squeeze(0),
            'intent_id': torch.tensor(categories[row["csType"].lower()], dtype=torch.long)
        }

    def __len__(self):
        return len(self.data)

In [39]:
train_dataset = DialoGPTDataset(training_data)
test_dataset = DialoGPTDataset(testing_data)
validation_dataset = DialoGPTDataset(validation_data)

print(len(train_dataset))
print(len(test_dataset))
print(len(validation_dataset))

print(train_dataset[0])


9532
2971
1470
{'input_ids': tensor([  101,  2672,  1996,  4895,  2071,  2831,  2000,  2216,  4004,  1998,
         3060,  3741,  3625,  2005,  3938,  1009,  1997,  1996, 10796,  1999,
         1996, 17401,  2612,  1997, 22604,  2006,  2023, 14636,  2055,  4785,
         2689,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,

In [40]:
class FeatureEncoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, next_input):
        super(FeatureEncoder, self).__init__()
        self.model = AutoModel.from_pretrained('GroNLP/hateBERT')
        self.hidden_dim = hidden_dim
        self.output_size = next_input

        self.informative_head = torch.nn.Sequential(
            torch.nn.Linear(self.hidden_dim, self.output_size),
            torch.nn.ReLU(),
        )

        self.questioning_head = torch.nn.Sequential(
            torch.nn.Linear(self.hidden_dim, self.output_size),
            torch.nn.ReLU(),
        )

        self.denouncing_head = torch.nn.Sequential(
            torch.nn.Linear(self.hidden_dim, self.output_size),
            torch.nn.ReLU(),
        )

        self.positive_head = torch.nn.Sequential(
            torch.nn.Linear(self.hidden_dim, self.output_size),
            torch.nn.ReLU(),
        )

        self.humor_head = torch.nn.Sequential(
            torch.nn.Linear(self.hidden_dim, self.output_size),
            torch.nn.ReLU(),
        )


    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        hate_speech_h = outputs.last_hidden_state[:, 0, :]

        informative_e = self.informative_head(hate_speech_h)
        questioning_e = self.questioning_head(hate_speech_h)
        denouncing_e = self.denouncing_head(hate_speech_h)
        positive_e = self.positive_head(hate_speech_h)
        humor_e = self.humor_head(hate_speech_h)

        return informative_e, questioning_e, denouncing_e, positive_e, humor_e, hate_speech_h

In [41]:
feature_encoder = FeatureEncoder(input_dim=128, hidden_dim=768, next_input=100)
print(feature_encoder(train_dataset[0]['input_ids'].unsqueeze(0), train_dataset[0]['attention_mask'].unsqueeze(0)))

(tensor([[0.2036, 0.2589, 0.2117, 0.0000, 0.0000, 0.1825, 0.0000, 0.6582, 0.2199,
         0.1190, 0.0000, 0.4363, 0.0806, 0.3400, 0.0000, 0.0000, 0.2178, 0.1873,
         0.0000, 0.0000, 0.5286, 0.0000, 0.0000, 0.1327, 0.0000, 0.6738, 0.0000,
         0.0000, 0.4698, 0.0309, 0.2999, 0.2617, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0980, 0.0069, 0.0000, 0.6666, 0.0000, 0.3196, 0.0000, 0.0000,
         0.0888, 0.0000, 0.3308, 0.2285, 0.0000, 0.3255, 0.0179, 0.0000, 0.3863,
         0.2729, 0.0000, 0.0000, 0.0000, 0.0000, 0.4242, 0.4294, 0.4014, 0.0000,
         0.0000, 0.6316, 0.1419, 0.0000, 0.1358, 0.1812, 0.0000, 0.0092, 0.2117,
         0.0000, 0.0404, 0.0088, 0.1594, 0.0000, 0.0000, 0.3304, 0.0876, 0.0502,
         0.2569, 0.1457, 0.0000, 0.7729, 0.2514, 0.1004, 0.0000, 0.0000, 0.1880,
         0.1814, 0.0000, 0.0000, 0.9527, 0.0725, 0.4245, 0.0000, 0.0000, 0.0247,
         0.0000]], grad_fn=<ReluBackward0>), tensor([[0.0000, 0.2653, 0.0000, 0.4704, 0.0610, 0.0000, 0.1222

In [42]:
class CounterSpeechNetwork(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, encoder_output, max_length):
        super(CounterSpeechNetwork, self).__init__()

        self.feature_encoder = FeatureEncoder(input_dim, hidden_dim, encoder_output)

        self.informative_decoder = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
        self.questioning_decoder = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
        self.denouncing_decoder = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
        self.positive_decoder = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
        self.humor_decoder = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

        self.informative_fusion = torch.nn.Linear(hidden_dim + encoder_output, self.informative_decoder.config.d_model)
        self.questioning_fusion = torch.nn.Linear(hidden_dim + encoder_output, self.questioning_decoder.config.d_model)
        self.denouncing_fusion = torch.nn.Linear(hidden_dim + encoder_output, self.denouncing_decoder.config.d_model)
        self.positive_fusion = torch.nn.Linear(hidden_dim + encoder_output, self.positive_decoder.config.d_model)
        self.humor_fusion = torch.nn.Linear(hidden_dim + encoder_output, self.humor_decoder.config.d_model)

        self.tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
        self.max_length = max_length

    def forward(self, input_ids, attention_mask, intent_id, counter_speech=None):
        informative_e, questioning_e, denouncing_e, positive_e, humor_e, hate_speech_h = self.feature_encoder(input_ids, attention_mask)

        batch_size = input_ids.size(0)

        fused = torch.zeros(batch_size, 1, self.informative_decoder.config.d_model, device=input_ids.device)

        for i in range(batch_size):
            if intent_id[i] == 0:
                fused[i] = self.informative_fusion(torch.cat((hate_speech_h[i], informative_e[i]), dim=-1)).unsqueeze(0)
            elif intent_id[i] == 1:
                fused[i] = self.questioning_fusion(torch.cat((hate_speech_h[i], questioning_e[i]), dim=-1)).unsqueeze(0)
            elif intent_id[i] == 2:
                fused[i] = self.denouncing_fusion(torch.cat((hate_speech_h[i], denouncing_e[i]), dim=-1)).unsqueeze(0)
            elif intent_id[i] == 3:
                fused[i] = self.positive_fusion(torch.cat((hate_speech_h[i], positive_e[i]), dim=-1)).unsqueeze(0)
            elif intent_id[i] == 4:
                fused[i] = self.humor_fusion(torch.cat((hate_speech_h[i], humor_e[i]), dim=-1)).unsqueeze(0)
            else:
                raise ValueError(f"Invalid intent_id: {intent_id[i]}")

        if counter_speech is not None:
            losses = []
            for i in range(batch_size):
                if intent_id[i] == 0:
                    output = self.informative_decoder(encoder_outputs=BaseModelOutput(last_hidden_state=fused[i].unsqueeze(0)), labels=counter_speech[i].unsqueeze(0))
                elif intent_id[i] == 1:
                    output = self.questioning_decoder(encoder_outputs=BaseModelOutput(last_hidden_state=fused[i].unsqueeze(0)), labels=counter_speech[i].unsqueeze(0))
                elif intent_id[i] == 2:
                    output = self.denouncing_decoder(encoder_outputs=BaseModelOutput(last_hidden_state=fused[i].unsqueeze(0)), labels=counter_speech[i].unsqueeze(0))
                elif intent_id[i] == 3:
                    output = self.positive_decoder(encoder_outputs=BaseModelOutput(last_hidden_state=fused[i].unsqueeze(0)), labels=counter_speech[i].unsqueeze(0))
                elif intent_id[i] == 4:
                    output = self.humor_decoder(encoder_outputs=BaseModelOutput(last_hidden_state=fused[i].unsqueeze(0)), labels=counter_speech[i].unsqueeze(0))
                losses.append(output.loss)
            avg_loss = sum(losses) / len(losses)  # Average loss across the batch
            return None, avg_loss  # No decoded text during training
        else:
            decoded_texts = []
            for i in range(batch_size):
                if intent_id[i] == 0:
                    output = self.informative_decoder.generate(encoder_outputs=BaseModelOutput(last_hidden_state=fused[i].unsqueeze(0)), max_length=self.max_length, num_beams=4, early_stopping=True)
                elif intent_id[i] == 1:
                    output = self.questioning_decoder.generate(encoder_outputs=BaseModelOutput(last_hidden_state=fused[i].unsqueeze(0)), max_length=self.max_length, num_beams=4, early_stopping=True)
                elif intent_id[i] == 2:
                    output = self.denouncing_decoder.generate(encoder_outputs=BaseModelOutput(last_hidden_state=fused[i].unsqueeze(0)), max_length=self.max_length, num_beams=4, early_stopping=True)
                elif intent_id[i] == 3:
                    output = self.positive_decoder.generate(encoder_outputs=BaseModelOutput(last_hidden_state=fused[i].unsqueeze(0)), max_length=self.max_length, num_beams=4, early_stopping=True)
                elif intent_id[i] == 4:
                    output = self.humor_decoder.generate(encoder_outputs=BaseModelOutput(last_hidden_state=fused[i].unsqueeze(0)), max_length=self.max_length, num_beams=4, early_stopping=True)
                decoded_texts.append(self.tokenizer.decode(output[0], skip_special_tokens=True))
            return decoded_texts, None  # Decoded text during inference, no loss

In [43]:
train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)

counterspeechnetwork = CounterSpeechNetwork(input_dim=128, hidden_dim=768, encoder_output=256, max_length=50)

for batch in train_dataloader:
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    intent_id = batch['intent_id']
    counter_speech = batch['counter_speech']

    print(counterspeechnetwork(input_ids, attention_mask, intent_id, counter_speech))
    break

(None, tensor(16.7536, grad_fn=<DivBackward0>))


In [44]:
model = CounterSpeechNetwork(input_dim=128, hidden_dim=768, encoder_output=256, max_length=50)
optimizer = AdamW(model.parameters(), lr=5e-5)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=32, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 10

for epoch in range(epochs):
    model.train()
    total_train_loss = 0.0

    # Wrap train_dataloader with tqdm for training progress
    train_loop = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{epochs} [Train]", leave=False)
    for batch in train_loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        counter_speech = batch['counter_speech'].to(device)
        intent_ids = batch['intent_id'].to(device)

        optimizer.zero_grad()

        # Process the entire batch at once
        _, loss = model(input_ids, attention_mask, intent_ids, counter_speech)

        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

        # Update tqdm with current batch loss
        train_loop.set_postfix({'batch_loss': loss.item(), 'avg_loss': total_train_loss / (train_loop.n + 1)})

    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}/{epochs} | Train Loss: {avg_train_loss:.4f}")

    # Validation
    model.eval()
    total_val_loss = 0.0

    # Wrap validation_dataloader with tqdm for validation progress
    val_loop = tqdm(validation_dataloader, desc=f"Epoch {epoch + 1}/{epochs} [Validation]", leave=False)
    with torch.no_grad():
        for batch in val_loop:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            counter_speech = batch['counter_speech'].to(device)
            intent_ids = batch['intent_id'].to(device)

            # Process the entire batch at once
            _, loss = model(input_ids, attention_mask, intent_ids, counter_speech)

            total_val_loss += loss.item()

            # Update tqdm with current batch loss
            val_loop.set_postfix({'batch_loss': loss.item(), 'avg_loss': total_val_loss / (val_loop.n + 1)})

    avg_val_loss = total_val_loss / len(validation_dataloader)
    print(f"Epoch {epoch + 1}/{epochs} | Validation Loss: {avg_val_loss:.4f}")

Epoch 1/1 | Train Loss: 2.2159


Epoch 1/1 | Validation Loss: 0.7556


In [45]:
torch.save(model.state_dict(), 'architecture_parameters.pth')

In [46]:
# Test evaluation
model.eval()
test_predictions = []
test_references = []

# Wrap test_dataloader with tqdm for test progress
test_loop = tqdm(test_dataloader, desc="Test Evaluation", leave=True)
with torch.no_grad():
    for batch in test_loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        counter_speech = batch['counter_speech'].to(device)  # Reference texts
        intent_ids = batch['intent_id'].to(device)

        # Generate predictions without providing counter_speech
        predictions, _ = model(input_ids, attention_mask, intent_ids)

        # Decode reference texts
        references = [model.tokenizer.decode(cs, skip_special_tokens=True) for cs in counter_speech]

        test_predictions.extend(predictions)
        test_references.extend(references)

        # Update tqdm with progress stats (e.g., number of predictions)
        test_loop.set_postfix({'predictions': len(test_predictions)})

# Final output
print(f"Generated {len(test_predictions)} test predictions")

Test Evaluation:  39%|██████████████████▉                              | 36/93 [03:55<06:13,  6.55s/it, predictions=1152]


KeyboardInterrupt: 

In [47]:
P, R, F1 = bert_score(test_predictions, test_references, lang="en", verbose=True)

# Final output
print(f"Generated {len(test_predictions)} test predictions")
print(f"BERTScore - Precision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/19 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/18 [00:00<?, ?it/s]

done in 3.01 seconds, 382.33 sentences/sec
Generated 1152 test predictions
BERTScore - Precision: 0.8691, Recall: 0.8621, F1: 0.8655


In [51]:
rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
for pred, ref in zip(test_predictions, test_references):
    scores = scorer.score(ref, pred)  # ROUGE scores for each prediction-reference pair
    rouge_scores['rouge1'].append(scores['rouge1'].fmeasure)
    rouge_scores['rouge2'].append(scores['rouge2'].fmeasure)
    rouge_scores['rougeL'].append(scores['rougeL'].fmeasure)

# Compute average ROUGE scores
avg_rouge1 = sum(rouge_scores['rouge1']) / len(rouge_scores['rouge1'])
avg_rouge2 = sum(rouge_scores['rouge2']) / len(rouge_scores['rouge2'])
avg_rougeL = sum(rouge_scores['rougeL']) / len(rouge_scores['rougeL'])

print(f"ROUGE - Rouge1: {avg_rouge1:.4f}, Rouge2: {avg_rouge2:.4f}, RougeL: {avg_rougeL:.4f}")

ROUGE - Rouge1: 0.2128, Rouge2: 0.0529, RougeL: 0.1574


In [50]:
for pred, ref in zip(test_predictions, test_references):
    print(f'Generated: {pred}')
    print(f'Actual: {ref}')
    print()

Generated: Every individual, regardless of their sexual orientation, deserves respect and equal rights. discrimination based on sexual orientation is unacceptable and goes against the principles of equality and respect.
Actual: It is false and deeply unfair to blame sick migrants for the problems faced by our country. we should focus on empathy, support, and effective healthcare systems for everyone.

Generated: Every individual, regardless of their sexual orientation, deserves equal rights and opportunities. discrimination based on sexual orientation is unacceptable and goes against the principles of equality and respect for all individuals.
Actual: Gender equality is a fundamental principle of human rights. women have the right to vote and should be treated equally to men in all aspects of life.

Generated: It is important to address misconceptions. muslims are a part of our society and contribute to our society in various fields such as science, arts, and philanthropy. they have mad